# Installation

In [ ]:
!pip install pandas
!pip install numpy
!pip install sqlalchemy
!pip install pymysql

Import libraries

In [4]:
import pandas as pd
import numpy as np
import sqlalchemy as db

# Extract

Read CSV files

In [5]:
appointments = pd.read_csv('smd-ns-appointments/appointments.csv')
clinics = pd.read_csv('smd-ns-appointments/clinics.csv', encoding='unicode_escape')
doctors = pd.read_csv('smd-ns-appointments/doctors.csv', encoding='unicode_escape')
px = pd.read_csv('smd-ns-appointments/px.csv', encoding='unicode_escape')

C:\Users\Kyle Carlo C. Lasala\AppData\Local\Temp\ipykernel_13904\3525776238.py:4: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  px = pd.read_csv('smd-ns-appointments/px.csv', encoding='unicode_escape')


# Transform

### Cleaning clinics.csv

Exploration

In [5]:
# preview
clinics

,clinicid,hospitalname,IsHospital,City,Province,RegionName
0,77EE3BC58CE560B86C2B59363281E914,St. Luke's Medical Center-Global City,True,Taguig,Manila,National Capital Region (NCR)
1,98C39996BF1543E974747A2549B3107C,Our Lady of Lourdes Hospital,True,Manila,Manila,National Capital Region (NCR)
2,9AEADE7BEADA35C83D3B344FBAFE43B0,Makati Medical Center,True,Makati,Manila,National Capital Region (NCR)
3,FDBD31F2027F20378B1A80125FC862DB,Cardinal Santos Medical Center,True,San Juan,Manila,National Capital Region (NCR)
4,205C3608ECB984C1F5F5D2F52C934428,NaN,False,Burgos,Ilocos Sur,Ilocos Region (I)
...,...,...,...,...,...,...
53957,8DE279A56DBCECE9F9FFC514A7D5A378,NaN,False,Manila,Manila,National Capital Region (NCR)
53958,55A64961C9AA4134016786AE7202682E,NaN,False,Manila,Manila,National Capital Region (NCR)
53959,075E464A7D15E6E5B9D8F8F5B5B16BB9,NaN,False,Manila,Manila,National Capital Region (NCR)
53960,01063BCF7624297FBB408495BCB62904,Malabon Hospital and Medical Center,True,Malabon,Manila,National Capital Region (NCR)


In [6]:
# check data type and null values
print(clinics.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53962 entries, 0 to 53961
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   clinicid      53962 non-null  object
 1   hospitalname  17538 non-null  object
 2   IsHospital    53962 non-null  bool  
 3   City          53962 non-null  object
 4   Province      53962 non-null  object
 5   RegionName    53962 non-null  object
dtypes: bool(1), object(5)
memory usage: 2.1+ MB
None


In [10]:
# check for duplicate ids
print("Duplicate IDs:", clinics['clinicid'].duplicated(keep=False).any())

Duplicate IDs: False


Checking Hospital Names

In [11]:
# check hospital names
hospital_names = clinics['hospitalname'].value_counts().reset_index().rename({"index":"hospital name", "hospitalname":"count"}, axis=1)
hospital_names = hospital_names.sort_values('hospital name').reset_index().drop('index', axis=1)
hospital_names

,hospital name,count
0,A. P. Cruz Community Hospital,28
1,A. Zarate General Hospital,11
2,ACE Dumaguete Doctors,37
3,ACE Medical Center - Butuan,11
4,ACE Medical Center - Cebu,28
...,...,...
1122,Zamboanga Puericulture Center Maternity Lying-...,2
1123,Zamboanga Sibugay Provincial Hospital,2
1124,Zamboanga del Norte Medical Center,5
1125,Zamboanga del Sur Medical Center,2


In [12]:
# Aggregating data from 'hospital_names' in chunks of 50 records to create a DataFrame 'check_hospital'
check_hospital = pd.DataFrame()
temp = pd.DataFrame()
counter = 0
for each in hospital_names.iloc:
    new = pd.DataFrame({"hospital name":[each['hospital name']], "count":[each['count']]})
    temp = pd.concat([temp, new])
    counter += 1
    if counter % 50 == 0:
        check_hospital = pd.concat([check_hospital, temp], axis=1)
        temp = pd.DataFrame()

In [13]:
#Resetting the index and dropping the 'index' column
check_hospital.reset_index().drop('index', axis=1).to_excel('hospital_names.xlsx')
check_hospital.reset_index().drop('index', axis=1)

,hospital name,count,hospital name,count,hospital name,count,hospital name,count,hospital name,count,...,hospital name,count,hospital name,count,hospital name,count,hospital name,count,hospital name,count
0,A. P. Cruz Community Hospital,28,Allied Care Experts (ACE) Medical Center - Pat...,54,"Balibago Polyclinic and Hospital, Inc.",8,Bulacan Maternity and Children's Hospital,1,Catalan Medical Hospital,3,...,Sacred Heart Hospital,9,Santo Tomas General Hospital,2,St. James Hospital,3,TMC Clinic @ Market Market,1,"Tobias-Feliciano Faith General Hospital, Inc.",5
1,A. Zarate General Hospital,11,Allied Care Experts (ACE) Medical Center - Que...,66,Balindong District Hospital,3,Bulacan Medical Center,15,Catanduanes Doctors Hospital,13,...,"Sacred Heart Hospital of Malolos, Inc.",93,"Santos General Hospital of Malolos, Inc.",4,"St. James Hospital, Inc.",30,TMC Clinic @ Regis Center Katipunan,1,Tojong Inc. Maternity and General Hospital,1
2,ACE Dumaguete Doctors,37,Allied Care Experts (ACE) Medical Center - Sar...,6,Balitucan District Hospital,1,Bulacan Medical Mission Group Pharmacy and Hos...,9,Cauayan District Hospital,1,...,Sacred Heart Medical Center,38,Sarangani Municipal Hospital,1,St. John Hospital,2,TMC Clinic @ Robinsons Magnolia,2,Toledo City General Hospital,4
3,ACE Medical Center - Butuan,11,Allied Care Experts (ACE) Medical Center - Tac...,15,Bangued Christian Hospital,2,"Bulacan Polymedic Hospital, Inc.",2,Cauayan Family Hospital,2,...,Sagrada Familia Hospital,3,Schistosomiasis Control and Research Hospital,3,"St. John Hospital, Inc.",1,TMC Clinic @ SM Fairview,1,Tomas-Cacal Medical Clinic and Hospital,1
4,ACE Medical Center - Cebu,28,Allied Care Experts (ACE) Medical Center - Val...,40,Basa Air Base Hospital,1,Buluan District Hospital,1,Cauayan Medical Specialist Hospital,5,...,Sagrado Medical & Children's Clinic,1,Seamen's Hospital,4,"St. John the Baptist Medical Center, Inc",20,TMC Clinic @ SM Light Mall,3,Tondo General Hospital and Medical Center,9
5,ACE Medical Center CDO,18,Almora General Hospital,1,Bataan Doctors Hospital,17,Bulusan Family Clinic,1,Cavite Center for Mental Health,2,...,Saint Agnes Hospital Center,3,Sen. Gerardo Roxas Memorial District Hospital,1,St. Joseph Southern Bukidnon Hospital,2,TMC Clinic @ SM Lipa,1,Tricity Medical Center,57
6,ACE Medical Center Legazpi,1,Alterado General Hospital,4,Bataan General Hospital,8,Bunawan District Hospital,1,"Cavite Doctors' Medical Center, Inc.",2,...,"Saint Pio's Medical Center, Inc.",5,Senor Sto. Nino Hospital-Bayambang,3,St. Jude Family Hospital,8,TMC Clinic @ SM Manila,1,"Trinity Woman Child Center The Birthplace""""",2
7,ACE Medical Center Palawan,10,Amai Pakpak Medical Center,17,Bataan Peninsula Medical Center,5,Bustos Community Hospital,3,Cavite East Asia Medical Center,26,...,Saint Threse Hospital,1,Severo Verallo Memorial District Hospital,2,St. Jude Hospital,3,TMC Clinic @ SM Marikina,1,Tubod Community Hospital,2
8,ACE Medical Center- Bohol,20,Amang Rodriguez Medical Center,10,"Bataan St. Joseph Hospital, Inc.",15,Butuan Doctor's Hospital,52,Cavite Medical Center,8,...,Saint Vincent General Hospital,5,Señor Sto. Niño Hospital,4,St. Jude Hospital and Medical Center,10,TMC Clinic @ SM North Annex,1,Tuburan District Hospital,1
9,AGO General Hospital,17,Amio Community Primary Hospital,1,Batanes General Hospital,4,Butuan Maternity Hospital and Puericulture Center,2,Cebu (Velez) General Hospital,44,...,Salve Regina General Hospital Inc.,6,Siargao Island Medical Center,2,St. Jude Thaddeus General Hospital,5,TMC Clinic @ SM San Mateo,1,Tudela Municipal Hospital,1


Checking Locations (City, Province, Region)

In [14]:
# check city names
city_names = pd.DataFrame()
counter = 0
temp = []
for each in np.sort(clinics['City'].unique()):
    temp.append(each)
    counter += 1
    if counter % 40 == 0:
        city_names = pd.concat([city_names, pd.DataFrame(temp)], axis=1)
        temp = []
        
city_names

,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,Aborlan,Antipas,Balaoan,Bayombong,Cabadbaran City,Carcar City,Dasmariñas City,Gingoog City,Jolo,Lasam,Maddela,Maria,Naguilian,Pamplona,Porac,San Isidro,Sara,Tagoloan
1,Abra de Ilog,Antipolo City,Balatan,Bien Unido,Cabagan,Cardona,Dauis,Gloria,Jose Dalman,Legazpi City,Madrid,Marihatag,Naic,Panabo City,Poro,San Jacinto,Sarangani,Tagudin
2,Abucay,Apalit,Balayan,Bilar,Cabanatuan City,Carigara,Davao City,Goa,Kabacan,Lemery,Magalang,Marikina,Narvacan,Panay,Pototan,San Joaquin,Sariaya,Taguig
3,Abulug,Aparri,Baler,Biliran,Cabarroguis,Carmen,Del Gallego,Gonzaga,Kabankalan City,Leon,Magallanes,Marilao,Nasipit,Pandan,Pozzorubio,San Jose,Sasmuan,Tagum City
4,Abuyog,Araceli,Balindong,Binalbagan,Cabatuan,Carmona,Diadi,Governor Generoso,Kabasalan,Leyte,Magdalena,Mariveles,Nasugbu,Pandi,President Roxas,San Jose City,Science City of Muñoz,Talavera
5,Adams,Arayat,Balingasag,Binalonan,Cabiao,Carranglan,Digos City,Guagua,Kalamansig,Lezo,Magsingal,Masantol,Natividad,Panganiban,Prosperidad,San Jose del Monte City,Siaton,Talisay
6,Agdangan,Argao,Baliuag,Binangonan,Cabugao,Casiguran,Dilasag,Gubat,Kalibo,Lian,Majayjay,Masbate City,Naval,Pangil,Puerto Galera,San Juan,Sibonga,Talisay City
7,Aglipay,Aringay,Baloi,Binmaley,Cabusao,Castillejos,Dimasalang,Guiguinto,Kalilangan,Libagon,Makati,Masinloc,Navotas,Panglao,Puerto Princesa City,San Leonardo,Sibulan,Talisayan
8,Agoncillo,Aritao,Bamban,Bislig City,Cabuyao City,Catanauan,Dimataling,Guihulngan City,Kananga,Libjo (Albor),Makilala,Maslog,New Bataan,Paniqui,Pulilan,San Luis,Silang,Tampakan
9,Agoo,Aroroy,Bambang,Biñan City,Cadiz City,Catarman,Dinagat,Guimba,Kapalong,Libmanan,Malabon,Mataasnakahoy,New Corella,Pantukan,Quezon,San Manuel,Silay City,Tanauan City


In [15]:
# check for province
prov_names = pd.DataFrame()
counter = 0
temp = []
for each in np.sort(clinics['Province'].unique()):
    temp.append(each)
    counter += 1
    if counter % 20 == 0:
        prov_names = pd.concat([prov_names, pd.DataFrame(temp)], axis=1)
        temp = []
        
prov_names

,0,0,0,0
0,Abra,Camarines Sur,Kalinga,Palawan
1,Agusan del Norte,Camiguin,La Union,Pampanga
2,Agusan del Sur,Capiz,Laguna,Pangasinan
3,Aklan,Catanduanes,Lanao del Norte,Quezon
4,Albay,Cavite,Lanao del Sur,Quirino
5,Angeles,Cebu,Leyte,Rizal
6,Antique,Compostela Valley,Maguindanao,Romblon
7,Apayao,Cotabato,Manila,Samar
8,Aurora,Davao Occidental,Marinduque,Sarangani
9,Basilan,Davao Oriental,Masbate,Siquijor


In [16]:
# Check for Region Name
pd.DataFrame(np.sort(clinics['RegionName'].unique()))

,0
0,Bangsamoro Autonomous Region in Muslim Mindana...
1,Bicol Region (V)
2,CALABARZON (IV-A)
3,Cagayan Valley (II)
4,Caraga (XIII)
5,Central Luzon (III)
6,Central Visayas (VII)
7,Cordillera Administrative Region (CAR)
8,Davao Region (XI)
9,Eastern Visayas (VIII)


In [17]:
# get the sorted list of unique region names
np.sort(clinics['RegionName'].unique())

array(['Bangsamoro Autonomous Region in Muslim Mindanao (BARMM)',
       'Bicol Region (V)', 'CALABARZON (IV-A)', 'Cagayan Valley (II)',
       'Caraga (XIII)', 'Central Luzon (III)', 'Central Visayas (VII)',
       'Cordillera Administrative Region (CAR)', 'Davao Region (XI)',
       'Eastern Visayas (VIII)', 'Ilocos Region (I)', 'MIMAROPA (IV-B)',
       'National Capital Region (NCR)', 'Northern Mindanao (X)',
       'SOCCSKSARGEN (Cotabato Region) (XII)', 'Western Visayas (VI)',
       'Zamboanga Peninsula (IX)'], dtype=object)

Checking ID Validity

In [235]:
# check if all ids are valid
clinics[[(len(each) != 32) for each in clinics['clinicid']]]

,clinicid,hospitalname,IsHospital,City,Province,RegionName


### Cleaning doctors.csv

Exploration

In [6]:
# preview
doctors

,doctorid,mainspecialty,age
0,AD61AB143223EFBC24C7D2583BE69251,General Medicine,41.0
1,D09BF41544A3365A46C9077EBB5E35C3,Family Medicine,43.0
2,FBD7939D674997CDB4692D34DE8633C4,Vascular Medicine,26.0
3,28DD2C7955CE926456240B2FF0100BDE,Otolaryngologists,34.0
4,35F4A8D465E6E1EDC05F3D8AB658C551,General Dentistry,50.0
...,...,...,...
60019,CD532DBEF6547A66D2138FAB49AA3B94,General Practitioner,33.0
60020,4473D870B5E31FAA40D2C45E1FF6DC27,NaN,NaN
60021,A4F554EB2C0934E7FDE2511E8C1573BA,NaN,NaN
60022,E540A361D93D37A33BB2F55D43DA79D9,General Practitioner,NaN


In [7]:
# check data type and null values
doctors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60024 entries, 0 to 60023
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   doctorid       60024 non-null  object 
 1   mainspecialty  27175 non-null  object 
 2   age            20028 non-null  float64
dtypes: float64(1), object(2)
memory usage: 1.4+ MB


In [527]:
# check for duplicate ids
print("Duplicate IDs:", doctors['doctorid'].duplicated(keep=False).any())

Duplicate IDs: False


Preprocessing Specialty

In [21]:
# remove null values in specialty
doctors['mainspecialty'] = doctors['mainspecialty'].fillna('none')

In [22]:
# get the list of specialties and their counts
specialty_list = doctors['mainspecialty'].value_counts().reset_index().rename({'index':'specialty', 'mainspecialty':'count'},axis=1)
specialty_list = specialty_list.sort_values('count', ascending=False).reset_index().drop('index', axis=1)
specialty_list

,specialty,count
0,none,32882
1,Internal Medicine,3812
2,General Medicine,2317
3,Pediatrics,1709
4,Family Medicine,894
...,...,...
3969,Mental Health / Addiction Medicine / Medico-Le...,1
3970,Family Medicine (Adult and Children Diseases),1
3971,GENERAL MEDICINE,1
3972,Nephrologu,1


In [23]:
# check for wrong case
specialty_list['specialty'] = specialty_list['specialty'].str.lower()
specialty_list.groupby('specialty').count()

,count
specialty,
\northopedic surgery,1
,1
cardiologist,1
adult cardiology,1
adult gastroenterology,1
...,...
yy,1
zxczcxcxzcxz,1
zzz,1


In [24]:
# fix wrong case and replace symbol
new_string = []
doctors['mainspecialty'] = doctors['mainspecialty'].str.lower()
for each in doctors['mainspecialty']:
    if '&' in each:
        print(each)
        noAmpSand = each.replace('&','and')
        new_string.append(noAmpSand)
    else:
        new_string.append(each)
doctors['mainspecialty'] = new_string

masters of science in preventive & regenerative medicine
obstetrics &gynecology
otolaryngology-head & neck surgery
ear nose throat - head & neck surgery (ent-hns)  rhinoplasty and facial plastic surgery
obstetrics & gynecology
otorhinolaryngologist-head & neck surgery/ent
oral & maxillofacial surgery
otolaryngology - head & neck surgery
general, cancer & laparoscopic surgery
general, cancer & laparoscopic-robotic surgery
obstetrics & gynecology
obstetrics & gynecology
ear, nose, throat - head & neck surgery
otorhinolaryngology - head & neck surgery
obstetrics & gynecology
physical medicine & rehabilitation
obstetrics & gynecology
dermatologist | dermatology | adult & pediatric conditions of the skin
obstetrics & gynecology
obstetrics & gynecology
obstetrics & gynecology
obstetrics & gynecology
general laparoscopic & cancer surgery
obstetrics & gynecology
ear, nose & throat - head and neck surgery
obstetrics & gynecology
developmental & behavioral pediatrics
general dentistry & orthodon

In [25]:
# reset the indices
specialty_list = doctors['mainspecialty'].value_counts().reset_index().rename({'index':'specialty', 'mainspecialty':'count'},axis=1)
specialty_list

,specialty,count
0,none,33006
1,internal medicine,4346
2,general medicine,2446
3,pediatrics,1781
4,family medicine,1072
...,...,...
3379,general surgery pediatric surgery,1
3380,"child and adult allergy, asthma and clinical i...",1
3381,dermatolgoy,1
3382,adult cardiology and internal medicine: heart ...,1


In [26]:
# check for end spaces and none
new_string = []
for each in doctors['mainspecialty']:
    i = 1
    if str(each) != 'nan':
        while (each[len(each)-i] == ' ' and i <= len(each)):
            endChar = each[len(each)-i]
            if endChar == ' ':
                i += 1
        j = 0
        if each != ' ':
            while (each[j] == ' ' and j < len(each)):
                j += 1
            if j > 1 or i > 1:
                print('0' + each[j:len(each)-(i-1)] + '0')
            if len(each) > 1:
                new_string.append(each[j:len(each)-(i-1)])
            else:
                new_string.append('none')
        else:
            new_string.append('none')
    else:
        new_string.append('none')

0orthodontist0
0neurosurgery0
0dentistry0
0general dentistry,orthodontics0
0pediatrics0
0general medicine0
0internal medicine0
0medical oncology0
0pediatric0
0surgery0
0orthodontics0
0internal medicine0
0dmd0
0esthetic0
0pediatrics0
0dermatology0
0obstetrics gynecology0
0general practitioner0
0ophthalmology0
0family medicine0
0otorhinolaryngology - head and neck surgery0
0surgery0
0surgery0
0refraction0
0internal medicine0
0internal medicine0
0orthopaedic spine surgery0
0neurosurgery0
0neurology0
0pulmonology0
0internal medicine0
0obstetrician gynecologist0
0adult cardiology0
0general medicine0
0hospice and palliative medicine0
0internal medicine0
0interventional cardiology/ adult cardiology/ internal medicine0
0general surgery and cancer surgery0
0pediatrics - pulmonology0
0adult cardiology0
0internal medicine0
0internal medicine0
0internal medicine-geriatrics0
0general physician0
0internal medicine0
0internal medicine0
0general physician0
0general medicine0
0general practitioner0
0ad

0general practitioner0
0primary care0
0ophthalmology0
0general practice0
0general practitioner0
0general practice0
0counseling0
0internal medicine0
0obstetrics and gynecology0
0dermatology0
0general physician0
0physical therapy0
0general practitioner0
0neurology0
0family medicine0
0general practice0
0counseling0
0aesthetic nurse0
0primary care and general medicine0
0general pediatrics0
0general physician0
0family medicine0
0small animal medicine0
0general physician0
0general medicine0
0general practice0
0counseling, stress management, anxiety management, depression, behavioral modification, assessment0
0general physician0
0general medicine0
0general practitioner0
0general physician0
0psychiatry0
0general practitioner0
0general medicine0
0general practitioner0
0general physician0
0general practitioner0
0dermatology0
0general physician0
0general medicine0
0general medicine0
0general physician0
0general physician0
0general physician0
0pediatrics0
0urology0
0internal medicine0


In [27]:
# fix string end spaces
doctors['mainspecialty'] = new_string
specialty_list = doctors['mainspecialty'].value_counts().reset_index().rename({'index':'specialty', 'mainspecialty':'count'},axis=1)
specialty_list

,specialty,count
0,none,33109
1,internal medicine,4511
2,general medicine,2516
3,pediatrics,1836
4,family medicine,1103
...,...,...
3179,marc,1
3180,orthopaedics (bone and joint surgery),1
3181,generalists,1
3182,"public health, pathology",1


In [28]:
# infer all internal medicine
new_string = []
for each in doctors['mainspecialty']:
    if 'internal med' == each[:12] or each == 'internal':
        new_string.append('internal medicine')
    else:
        new_string.append(each)

In [29]:
# reset the indices
doctors['mainspecialty'] = new_string
specialty_list = doctors['mainspecialty'].value_counts().reset_index().rename({'index':'specialty', 'mainspecialty':'count'},axis=1)
specialty_list

,specialty,count
0,none,33109
1,internal medicine,5263
2,general medicine,2516
3,pediatrics,1836
4,family medicine,1103
...,...,...
2905,"emergency medicine, primary care",1
2906,ob/rm check up,1
2907,helping mother to gave birth by normal delivery,1
2908,otorhinolaryngology-hns,1


In [30]:
# infer the category of 2 letter strings
new_string = []
valid_string_2 = {
    'internal medicine':['im','in'], 
    'general medicine':['gp','md'],
    'nursing':['rn'],
    'family medicine':['fm'],
    'physical medicine and rehabilitation':['pt'],
    'medical oncology':['on'],
    'obstetrics and gynecology':['ob','gy'],
    'general surgery':['gs'],
    'emergency medicine':['er'],
    'infectious disease':['id']
}
# counter = 0
# for each in specialty_list['specialty']:
#     if len(each) == 2:
#         print(counter, each)
#         counter += 1
for each in doctors['mainspecialty']:
    if len(each) == 2:
        appendNone = True
        for eachKey in valid_string_2:
            if each in valid_string_2[eachKey]:
                new_string.append(eachKey)
                appendNone = False
                break
        if appendNone:
            new_string.append('none')
    else:
        new_string.append(each)

In [31]:
# reset the indices
doctors['mainspecialty'] = new_string
specialty_list = doctors['mainspecialty'].value_counts().reset_index().rename({'index':'specialty', 'mainspecialty':'count'},axis=1)
specialty_list

,specialty,count
0,none,33283
1,internal medicine,5397
2,general medicine,2633
3,pediatrics,1836
4,obstetrics and gynecology,1160
...,...,...
2839,helping mother to gave birth by normal delivery,1
2840,otorhinolaryngology-hns,1
2841,newborn medicine,1
2842,"dermatology (skin, hair, nails)",1


In [32]:
# infer general medicine
for each in specialty_list['specialty']:
    if 'general medicine' == each[:16] or 'general' == each:
        print(each)
        print('-----')

general medicine
-----
general
-----
general medicine, primary care
-----
general medicine, occupational medicine
-----
general medicine, public health
-----
general medicine and primary care
-----
general medicine, occupational health
-----
general medicine / primary care
-----
general medicine | primary care
-----
general medicine and public health
-----
general medicine and surgery
-----
general medicine, occupational and industrial medicine
-----
general medicine, general surgery
-----
general medicine, anesthesiology
-----
general medicine, general practitioner
-----
general medicine and pediatrics
-----
general medicine/occupational medicine
-----
general medicine, obstetrics and gynecology
-----
general medicine/ occupational health
-----
general medicine, occupational medicine, and public health
-----
general medicine, primary care, preventive medicine
-----
general medicine, general practice
-----
general medicine - primary care
-----
general medicine, primary health care
----

In [33]:
# set to general medicine
new_string = []
for each in doctors['mainspecialty']:
    if 'general medicine' == each[:16] or 'general' == each:
        new_string.append('general medicine')
    else:
        new_string.append(each)

In [34]:
# reset the indices
doctors['mainspecialty'] = new_string
specialty_list = doctors['mainspecialty'].value_counts().reset_index().rename({'index':'specialty', 'mainspecialty':'count'},axis=1)
specialty_list

,specialty,count
0,none,33283
1,internal medicine,5397
2,general medicine,2921
3,pediatrics,1836
4,obstetrics and gynecology,1160
...,...,...
2715,orthopaedic surgery - hand specialist,1
2716,occupational medicine and public health,1
2717,"pediatric, allergies",1
2718,general pediatrics  pediatric infectious dise...,1


In [35]:
# infer pediatrician
for each in specialty_list['specialty']:
    if 'general pedia' == each[:13] or 'pedia' == each[:5]:
        print(each)
        print('-----')

pediatrics
-----
general pediatrics
-----
pediatrician
-----
pedia
-----
pediatric
-----
general pediatrician
-----
pediatric dentistry
-----
pediatrics and child health
-----
pediatric surgery
-----
pediatric neurology
-----
pediatric medicine
-----
pediatrics, general medicine
-----
pediatric cardiology
-----
pediatric pulmonology
-----
pediatrics - pulmonology
-----
pediatrics and general medicine
-----
general pediatric
-----
pediatics
-----
pediatric infectious diseases
-----
general pediatrics and pediatric hematology
-----
pediatric hematology-oncology
-----
pediatric gastroenterology, hepatology and nutrition
-----
pediatrics neurology
-----
pediatrics. family medicine
-----
pediatric therapy
-----
pediatric cardiologist
-----
pediatric hematology and oncology
-----
pediatrics/ occupational health physician
-----
pediatric neurology, general pediatrics
-----
pediatric neurology | general pediatrics | stroke
-----
pediatrics

pediatrics
-----
general pediatrics and medicine
----

In [36]:
# set to pediatrics
new_string = []
for each in doctors['mainspecialty']:
    if 'general pedia' == each[:13] or 'pedia' == each[:5]:
        new_string.append('pediatrics')
    else:
        new_string.append(each)

In [37]:
# reset the indices
doctors['mainspecialty'] = new_string
specialty_list = doctors['mainspecialty'].value_counts().reset_index().rename({'index':'specialty', 'mainspecialty':'count'},axis=1)
specialty_list

,specialty,count
0,none,33283
1,internal medicine,5397
2,general medicine,2921
3,pediatrics,2590
4,obstetrics and gynecology,1160
...,...,...
2594,"ear, nose and throat-head and neck surgery",1
2595,family medicine (diplomate),1
2596,im-cardiologist,1
2597,cecilia gonzales,1


In [38]:
# infer more internal medicine
for each in specialty_list['specialty']:
    if (each[:3] == 'im ' or each[:3] == 'im-' or each[:4] == 'im -') and not (each[:6] == 'im not') and not (each[:10] == 'im trained'):
        print(each)

im-nephrology
im-pulmonary medicine
im-cardiology
im-gastroenterology
im- cardiology
im cardiology
im nephro
im - pulmonary medicine
im - adult cardiology
im - cardiology - electrophysiology
im-pulmonary critical care
im - infectious diseases
im - adult cardiology and vascular medicine
im - hematology
im - nephrology
im-pulmonology
im - pulmonary and sleep medicine
im-nephrologist
im nephrology
im-gastroenterolgy
im - rheumatology
im-infectious diseases
im-endocrinology
im-medical oncology
im - oncology
im-hematology
im - adult cardiology / interventional cardiology
im- endocrinology
im- pulmonology
im-cardiologist


In [39]:
# set to internal medicine
new_string = []
for each in doctors['mainspecialty']:
    if (each[:3] == 'im ' or each[:3] == 'im-' or each[:4] == 'im -') and not (each[:6] == 'im not') and not (each[:10] == 'im trained'):
        new_string.append('internal medicine')
    else:
        new_string.append(each)

In [40]:
# reset the indices
doctors['mainspecialty'] = new_string
specialty_list = doctors['mainspecialty'].value_counts().reset_index().rename({'index':'specialty', 'mainspecialty':'count'},axis=1)
specialty_list

,specialty,count
0,none,33283
1,internal medicine,5445
2,general medicine,2921
3,pediatrics,2590
4,obstetrics and gynecology,1160
...,...,...
2564,"ear, nose, throat-head and neck surgery",1
2565,"ear, nose and throat-head and neck surgery",1
2566,family medicine (diplomate),1
2567,cecilia gonzales,1


In [41]:
# infer family medicine        
for each in specialty_list['specialty']:
    if 'family med' == each[:10]:
        print(each)

family medicine
family med
family medicine specialist
family medecine
family medicine,internal medicine
family medicine diabetologist
family medicine and occupational medicine; technical health and safety, life and mental health coach, process improvement, workplace legal health industry expert, accredited osh consultant and speaker
family medice
family medicine, internal medicine (in training), general medicine, public health
family medicine, adult medicine, occupational medicine
family medicine, lifestyle medicine
family medicne
family medicine under training
family medicine - infectious disease and tropical medicine (fm-idtm)
family medicine, ophthalmology
family medicine/ medical acupuncture
family medicine, cfp
family medicine and birthing home
family medicine, adult medicine
family medicine  oh pysician
family medicine, surgery, internal medicine,
family medicine (primary care practice in mental health, hypertension, diabetes, sleep)
family medicine (adult and children diseases)


In [42]:
# set to family medicine
new_string = []
for each in doctors['mainspecialty']:
    if 'family med' == each[:10]:
        new_string.append('family medicine')
    else:
        new_string.append(each)

In [43]:
# reset the indices
doctors['mainspecialty'] = new_string
specialty_list = doctors['mainspecialty'].value_counts().reset_index().rename({'index':'specialty', 'mainspecialty':'count'},axis=1)
specialty_list

,specialty,count
0,none,33283
1,internal medicine,5445
2,general medicine,2921
3,pediatrics,2590
4,family medicine,1175
...,...,...
2516,trisha reyes,1
2517,dog and cat medicine and surgery,1
2518,gynecologue,1
2519,hospital administration,1


In [44]:
# infer obgyne
for each in specialty_list['specialty']:
    if 'ob' == each[:2] and 'obesity' != each[:7]:
        print(each)
        print('-----')

obstetrics and gynecology
-----
ob-gyn
-----
obgyn
-----
ob gyn
-----
ob gyne
-----
ob-gyne
-----
obgyne
-----
obstetrics-gynecology
-----
obstetrician-gynecologist
-----
obstetrics gynecology
-----
obstetrician gynecologist
-----
obstetrics and gynaecology
-----
obstetrics
-----
ob gynecologist
-----
obstetrics - gynecology
-----
obstetrician and gynecologist
-----
obsterics and gynecology
-----
obstetric and gynecology
-----
obstetrician
-----
oby
-----
obstetrics and gynecology (obgyn)
-----
obstetrics and gynecologist
-----
ob/gyn
-----
obstetrician gynecology
-----
obstetrician - gynecologist
-----
ob-gynecologist
-----
obstetric
-----
ob/gyne
-----
ob_gyn
-----
obstetrics and gynecology (ob-gyn)
-----
ob gynecology
-----
obstetrician- gynecologist
-----
obsterics gynecology
-----
obstetrics and pediatrics
-----
obstetrics- gynecology
-----
obstetrics and gynecology - gynecologic oncology
-----
obstetrician/gynecologist
-----
ob- gyne
-----
ob sonoogist
-----
obgyne- perinatology


In [45]:
# set to obstetrics and gynecology
new_string = []
for each in doctors['mainspecialty']:
    if 'ob' == each[:2] and 'obesity' != each[:7]:
        new_string.append('obstetrics and gynecology')
    else:
        new_string.append(each)

In [46]:
# reset the indices
doctors['mainspecialty'] = new_string
specialty_list = doctors['mainspecialty'].value_counts().reset_index().rename({'index':'specialty', 'mainspecialty':'count'},axis=1)
specialty_list

,specialty,count
0,none,33283
1,internal medicine,5445
2,general medicine,2921
3,pediatrics,2590
4,obstetrics and gynecology,2077
...,...,...
2392,"neurology, stroke specialist",1
2393,"fellow pso-hns, ent-hns (voice, swallowing, no...",1
2394,aesthetic and anti-aging medicine,1
2395,"ear, nose throat-head and neck surgery",1


In [47]:
# infer neurology
for each in specialty_list['specialty']:
    if ('neuro' == each[:5] or 
    'adult neuro' == each[:11] or
    'child neuro' == each[:11] or
    "adult and pediatric neurology" == each
       ):
        print(each)
        print('-------')

neurology
-------
adult neurology
-------
neurosurgery
-------
neurology and psychiatry
-------
neurologist
-------
neuro
-------
child neurology
-------
neuroscience
-------
neurology - internal medicine
-------
neurosurgeon
-------
neuroloy
-------
neurology, psychiatry
-------
neurology - diseases of the brain, spinal cord, nerves and muscles
-------
neurologist/sleep medicine
-------
neurosurgery spinal
-------
neuropsychological assessment and neurodevelopmental evaluation, applied behavioral analysis intervention and behavioural support programs
-------
neurology (adult)
-------
adult neurology, general medicine, primary care
-------
neurology/parkinsons disease and movement disorders
-------
neuro surgeon
-------
neuro doctor
-------
child neurology and neurodevelopment
-------
neurology- psychiatry
-------
adult neurologist, neurocritical care and trauma specialist
-------
adult neurology and psychiatry
-------
child neurology and epilepsy specialist
-------
neurology | intern

In [48]:
# set to neurology
new_string = []
for each in doctors['mainspecialty']:
    if ('neuro' == each[:5] or 
    'adult neuro' == each[:11] or
    'child neuro' == each[:11] or
    "adult and pediatric neurology" == each
       ):
        new_string.append('neurology')
    else:
        new_string.append(each)

In [49]:
# reset the indices
doctors['mainspecialty'] = new_string
specialty_list = doctors['mainspecialty'].value_counts().reset_index().rename({'index':'specialty', 'mainspecialty':'count'},axis=1)
specialty_list

,specialty,count
0,none,33283
1,internal medicine,5445
2,general medicine,2921
3,pediatrics,2590
4,obstetrics and gynecology,2077
...,...,...
2355,welder pipe filter,1
2356,general practice/ maritime medicine,1
2357,"interventional radiology, vascular and interve...",1
2358,"fellow pso-hns, ent-hns (voice, swallowing, no...",1


In [50]:
# infer ent
valid_ent = ['ent ', ' ent', 'ent-', '-ent']
invalid_ent = ['patient - wrong entry', 'urgent care', 'ophthalmology and ent','bone alignment therapy',
              'wrong entry', 'child, adolescent and adult psychiatry and behavioral mdicine',
              'patient care','event coordinator','department of internal medicine','early childhood development | primary care',
              'government service','gastro enterologist','ent otolaryngology','dental practice management and leadership']
for each in specialty_list['specialty']:
    if each == 'ent':
        print(each)
        print('------')
    elif each in invalid_ent:
        continue
    else:
        for each2 in valid_ent:
            if each2 in each:
                print(each)
                print('------')
                break

ent
------
ent-hns
------
ent head and neck surgery
------
ent-head and neck surgery
------
ent - head and neck surgery
------
ent hns
------
otorhinolaryngology, head and neck surgery ( ent )
------
ent (otorhinolaryngology)
------
ent (otorhinolaryngology) - head and neck surgery
------
ent (otolaryngology)
------
ear, nose, throat, head and neck surgery (ent-hns)
------
ent otorhinolaryngology
------
ent head and neck surgeon
------
ent (otorhinolaryngology head and neck surgery)
------
ent (otorhinolaryngology) head and neck surgery
------
otorhinolaryngology (ears, nose and throat), ent-head and neck surgery,
------
ent - hns(ear,nose,throat head and neck surgery)
------
ent-hns( otorhinolaryngology)
------
ent-hns /otolaryngology-head and neck surgery
------
otorhinolaryngology - head and neck surgery (orl/ent-hns)
------
otolaryngology, head and neck specialty (ent-hns)
------
otolaryngology-head and neck surgery (ent-hns)
------
ent-head and neck specialist
------
otolaryngolog

In [51]:
# set to ent
new_string = []
for each in doctors['mainspecialty']:
    appendEach = True
    if each == 'ent':
        appendEach = False
        new_string.append('ent')
    elif each in invalid_ent:
        new_string.append(each)
        appendEach = False
        continue
    else:
        for each2 in valid_ent:
            if each2 in each:
                new_string.append('ent')
                appendEach = False
                break
    if appendEach:
        new_string.append(each)

In [52]:
# reset the indices
doctors['mainspecialty'] = new_string
specialty_list = doctors['mainspecialty'].value_counts().reset_index().rename({'index':'specialty', 'mainspecialty':'count'},axis=1)
specialty_list

,specialty,count
0,none,33283
1,internal medicine,5445
2,general medicine,2921
3,pediatrics,2590
4,obstetrics and gynecology,2077
...,...,...
2296,patienr,1
2297,breast surgery,1
2298,"general surgery, trauma surgery and critical care",1
2299,"endocrinology, diabetes mellitus, thyroid dise...",1


In [53]:
# infer more gen med (general practitioner)
invalid_gen_med = ['occupational medicine and general practice','occupational health physician, general practice, primary care physician',
                  'family and community medicine, general practice, adult and child medicine',
                  'mental health;  general practice - with training in psychiatry',
                  'mental health, general practice with training in psychiatry','dermatology, general practitioner',
                  'occupational medicine / general practitioner','public health and general practice',
                  'general surgery, general practice','occupational medicine / general practice','anesthesia, general practitioner',
                  'anesthesiology, general practitioner']
for each in specialty_list['specialty']:
    if 'general prac' in each:
        appendEach = True
        for each2 in invalid_gen_med:
            if each2 == each:
                appendEach = False
                break
        if appendEach:
            print(each)
            print('----')

general practitioner
----
general practice
----
general practicioner
----
general practioner
----
general practice, primary care
----
general practice, general medicine, primary care
----
general practitioners
----
general practitioner, general medicine
----
general practitioner, occupational health physician
----
general practice, occupational health
----
general practice/primary care
----
general practitioner, primary care and public health
----
general practiotioner
----
general practitioner; internal medicine
----
general practitioner, pain medicine
----
general practice, public health
----
general practitioner, pediatrics junior consultant
----
general practitioner medicine
----
general practitioner/general medicine
----
general practice / internal medicine
----
general practitioner, primary care, preventive care
----
md - general practitioner
----
general practice,  occupational medicine
----
general practitioner, occupational medicine
----
general practioner, pediatrics
----
gen

In [54]:
# set to general medicine
new_string = []
for each in doctors['mainspecialty']:
    if 'general prac' in each:
        appendEach = True
        for each2 in invalid_gen_med:
            if each2 == each:
                new_string.append(each)
                appendEach = False
                break
        if appendEach:
            new_string.append('general medicine')
    else:
        new_string.append(each)

In [55]:
# reset the indices
doctors['mainspecialty'] = new_string
specialty_list = doctors['mainspecialty'].value_counts().reset_index().rename({'index':'specialty', 'mainspecialty':'count'},axis=1)
specialty_list

,specialty,count
0,none,33283
1,internal medicine,5445
2,general medicine,4628
3,pediatrics,2590
4,obstetrics and gynecology,2077
...,...,...
2226,"cardiology, internal medicine",1
2227,web developer,1
2228,psychatry,1
2229,"ear, nose , throat - head and neck surgery",1


In [56]:
# infer cardiology
invalid_cardio = ['pulmonologist, cardiologist and internal medicine']
for each in specialty_list['specialty']:
    if 'cardio' in each:
        appendEach = True
        for each2 in invalid_cardio:
            if each2 == each:
                appendEach = False
                break
        if appendEach:
            print(each)
            print('---------')

cardiology
---------
adult cardiology
---------
cardiologist
---------
cardio
---------
interventional cardiology
---------
thoracic and cardiovascular surgery
---------
cardiothoracic surgery
---------
cardiovascular and thoracic surgery
---------
cardiovascular surgery
---------
adult cardiology - internal medicine
---------
cardiovascular
---------
electrophysiology/ cardiologist
---------
i'm-cardio
---------
thoracic and cardiovascular surgery, functional medicine
---------
adult cardiology -internal medicine
---------
internal-medicine- adult cardiology
---------
interventional cardiology / internal medicine
---------
cardiovascular perfusion
---------
cardiologist.
---------
cardiothoracic and vascular surgery
---------
interventional cardiologist
---------
cardiology / internal medicine
---------
thoracic cardiovascular surgery
---------
cardiologue
---------
adult interventional cardiology
---------
thoracic and cardiovascular surgery, general surgery
---------
cardio vascular

In [57]:
# set to cardiology
new_string = []
for each in doctors['mainspecialty']:
    if 'cardio' in each:
        appendEach = True
        for each2 in invalid_cardio:
            if each2 == each:
                new_string.append(each)
                appendEach = False
                break
        if appendEach:
            new_string.append('cardiology')
    else:
        new_string.append(each)

In [58]:
# reset the indices
doctors['mainspecialty'] = new_string
specialty_list = doctors['mainspecialty'].value_counts().reset_index().rename({'index':'specialty', 'mainspecialty':'count'},axis=1)
specialty_list

,specialty,count
0,none,33283
1,internal medicine,5445
2,general medicine,4628
3,pediatrics,2590
4,obstetrics and gynecology,2077
...,...,...
2184,welder pipe filter,1
2185,"interventional radiology, vascular and interve...",1
2186,aesthetic and anti-aging medicine,1
2187,"ear, nose throat-head and neck surgery",1


In [59]:
# infer more general medicine (general physician)
for each in specialty_list['specialty']:
    if 'general phy' in each and each[:3] != 'psy' and each[:3] != 'rad':
        print(each)
        print('-----')

general physician
-----
general physician, primary care
-----
general phycisian
-----
general physician (adults and pediatrics)
-----
general physician, surgeon
-----
general physician/ primary skin health physician / dermatology
-----
general physician / 09368241461
-----
general physician, nutrition and dietetics
-----
general physician, covid-19 suspect/confirmed
-----
general physician, occupational health physician
-----
primary care physician / general physician
-----
general physician, medical acupuncturist
-----
general physician, 1st year - practice-based family medicine resident
-----
general physician / 09559301258
-----
general physician
occupational health physician
-----
general physician/ general surgery
-----
general physician/pediatrics
-----
general phycians
-----
general physician/ occupational health physician
-----
general physician / occupational health medicine
-----
general physican
-----
general physician (pediatrics and adult)
-----


In [60]:
# set to general medicine
new_string = []
for each in doctors['mainspecialty']:
    if 'general phy' in each and each[:3] != 'psy' and each[:3] != 'rad':
        new_string.append('general medicine')
    else:
        new_string.append(each)

In [61]:
# reset the indices doctors['mainspecialty'] = new_string
specialty_list = doctors['mainspecialty'].value_counts().reset_index().rename({'index':'specialty', 'mainspecialty':'count'},axis=1)
specialty_list

,specialty,count
0,none,33283
1,general medicine,5644
2,internal medicine,5445
3,pediatrics,2590
4,obstetrics and gynecology,2077
...,...,...
2162,diagnostic and interventional radiology,1
2163,orthopaedic surgery - hand specialist,1
2164,occupational medicine and public health,1
2165,phd clinical psychology on going,1


In [62]:
# infer orthopedic surgery
for each in specialty_list['specialty']:
    if 'orthop' in each:
        print(each)
        print('----')

orthopedic surgery
----
orthopedics
----
orthopaedic surgery
----
orthopaedics
----
orthopedic
----
orthopedic surgery and traumatology
----
orthopedic surgeon
----
general orthopedics
----
orthopaedics and traumatology
----
orthopaedic
----
orthopedics and sports medicine
----
orthopedic and spine surgery
----
general orthopedic surgery
----
orthopaedics surgery
----
orthopaedic surgeon
----
orthopaedic surgery and traumatology
----
general orthopedics and traumatology
----
orthopedic spine surgery
----
orthopaedic spine surgery
----
orthopedic surgery + traumatology
----
manual, sports, orthopedic rehab
----
orthopedic surgery and trauma
----
orthopedic surgery - sports medicine and joint reconstruction surgery
----
general orthopedids
----
orthopedics/orthopedic surgery
----
orthopidic
----
orthopedics / orthopaedics, general orthopaedics
----
general orthopedics and trauma
----
general orthopaedics and trauma surgery, arthroscopic shoulder surgery and arthroplasty
----
general orth

In [63]:
# set to orthopedic surgery
new_string = []
for each in doctors['mainspecialty']:
    if 'orthop' in each:
        new_string.append('orthopedic surgery')
    else:
        new_string.append(each)

In [64]:
# reset the indices
doctors['mainspecialty'] = new_string
specialty_list = doctors['mainspecialty'].value_counts().reset_index().rename({'index':'specialty', 'mainspecialty':'count'},axis=1)
specialty_list

,specialty,count
0,none,33283
1,general medicine,5644
2,internal medicine,5445
3,pediatrics,2590
4,obstetrics and gynecology,2077
...,...,...
2097,dematology,1
2098,welder pipe filter,1
2099,"interventional radiology, vascular and interve...",1
2100,aesthetic and anti-aging medicine,1


In [65]:
# infer allergy 
for each in specialty_list['specialty']:
    if 'aller' in each and each != 'infectious disease, allergology':
        print(each)

allergy
allergology
allergies
adult and pediatric allergy and immunology
allergy, asthma, general and reproductive immunology
allergologist
allergy and reproductive immunology
allergology and immunology
allergy , asthma, immunology
adult and child allergy, asthma and clinical immunology
allergylogist
adult and pediatric asthma, allergy and immunology specialist
allergy and clinical immunology
allergology and immunology, general pediatrics
allergists
allergy / immunology
asthma and allergy
adult and pediatric allergy, asthma, and immunology
allergists/immunologists
adult and pediatric allergology - immunology / general pediatrics
child and adult allergy, asthma and clinical immunology
allergist


In [66]:
# set to allergy
new_string = []
for each in doctors['mainspecialty']:
    if 'aller' in each and each != 'infectious disease, allergology':
        new_string.append('allergy')
    else:
        new_string.append(each)

In [67]:
# reset the indices
doctors['mainspecialty'] = new_string
specialty_list = doctors['mainspecialty'].value_counts().reset_index().rename({'index':'specialty', 'mainspecialty':'count'},axis=1)
specialty_list

,specialty,count
0,none,33283
1,general medicine,5644
2,internal medicine,5445
3,pediatrics,2590
4,obstetrics and gynecology,2077
...,...,...
2076,"endocrinology, diabetes mellitus, thyroid dise...",1
2077,reumathology,1
2078,dematology,1
2079,welder pipe filter,1


In [68]:
# infer derma
for each in specialty_list['specialty']:
    if 'derma' in each and each[:6] != 'dental':
        print(each)
        print('-------')

dermatology
-------
dermatologist
-------
derma
-------
aesthetic dermatology
-------
dermatologists
-------
cosmetic dermatology
-------
dermatology and aesthetics
-------
clinical and aesthetic dermatology
-------
dermatology (adult and pediatric)
-------
medical dermatology
-------
dermatology and venereology
-------
dermatology (pds)-pathologic and cosmetic hair, skin and nail specialist
-------
dermatology, diseases of the skin, hair and nails, clinical and aesthetic dermatology
-------
dermatology resident
-------
dermatology/ pediatrics
-------
fellow, aesthetic dermatology, cosmetic surgery
-------
dermatology-hair transplant
-------
aesthetic medicine, cosmetic dermatology
-------
aesthetic dermatologist and anti-aging medicine
-------
derma specialist
-------
dermatologist (skin)
-------
aesthetic and pathologic dermatology
-------
clinical and cosmetic dermatology / occupational md - general med
-------
dermatology, pediatrics,
-------
derma1
-------
clinical, aesthetic and 

In [69]:
# set to dermatology
new_string = []
for each in doctors['mainspecialty']:
    if 'derma' in each and each[:6] != 'dental':
        new_string.append('dermatology')
    else:
        new_string.append(each)

In [70]:
# reset the indices
doctors['mainspecialty'] = new_string
specialty_list = doctors['mainspecialty'].value_counts().reset_index().rename({'index':'specialty', 'mainspecialty':'count'},axis=1)
specialty_list

,specialty,count
0,none,33283
1,general medicine,5644
2,internal medicine,5445
3,pediatrics,2590
4,obstetrics and gynecology,2077
...,...,...
2021,breast surgery,1
2022,"general surgery, trauma surgery and critical care",1
2023,"endocrinology, diabetes mellitus, thyroid dise...",1
2024,reumathology,1


In [71]:
# infer occupational medicine
for each in specialty_list['specialty']:
    if 'occupational' in each[:12]:
        print(each)
        print('-----')

occupational medicine
-----
occupational health
-----
occupational health physician
-----
occupational therapy
-----
occupational and industrial medicine
-----
occupational health medicine
-----
occupational
-----
occupational health, general medicine
-----
occupational and preventive medicine
-----
occupational and industrial health medicine
-----
occupational health physician; general medicine
-----
occupational and safety medicine
-----
occupational health and industrial medicine; anesthesiology and pain medicine
-----
occupational, speech and physical therapies
-----
occupational health physician, general practice, primary care physician
-----
occupational health, public health
-----
occupational health, primary care, public health
-----
occupational medicine and hypertension specialist
-----
occupational health medicine / general medicine
-----
occupational and industrial medicine, hypertension specialist, primary care
-----
occupational medicine and general practice
-----
occupat

In [72]:
# set to occupational medicine
new_string = []
for each in doctors['mainspecialty']:
    if 'occupational' in each[:12]:
        new_string.append('occupational medicine')
    else:
        new_string.append(each)

In [73]:
# reset the indices
doctors['mainspecialty'] = new_string
specialty_list = doctors['mainspecialty'].value_counts().reset_index().rename({'index':'specialty', 'mainspecialty':'count'},axis=1)
specialty_list

,specialty,count
0,none,33283
1,general medicine,5644
2,internal medicine,5445
3,pediatrics,2590
4,obstetrics and gynecology,2077
...,...,...
1983,reumathology,1
1984,dematology,1
1985,welder pipe filter,1
1986,"interventional radiology, vascular and interve...",1


In [74]:
# infer general dentistry
invalid_dentist = ['student','periodontics and implant dentistry','students','health optimization medicine, bioidentical hormone balancing (male and female), nutritional medicine',
                  'medical student','family and occupational medicine resident','emergency medicine resident',
                  'surgical resident','psychiatry resident','periodontics and dental implants','oral surgery - dental implants',
                  'anesthesiology and family medicine resident','med student','med-student','resident','oral surgery, prosthodontics, general dentistry',
                  'oral surgery, cosmetic dentist, endodontist', 'general surgery resident', 'oral surgery, orthodontics, cosmetic dentistry']
for each in specialty_list['specialty']:
    if 'dent' in each:
        appendEach = True
        for each2 in invalid_dentist:
            if each == each2:
                appendEach = False
                break
        if appendEach:
            print(each)
            print('=========')

general dentistry
dentistry
general dentist
dentist
dental medicine
dental
general dentistry, orthodontics
general dentistry and orthodontics
family dentistry
general dentistry and orthodontic
cosmetic dentistry
general dentistry, pediatric dentistry
general dentisry
aesthetic dentistry
general dentistry, orthodontics and oral surgery
general dentistry,
implant dentistry
dental surgery, orthodontics
general dentistry, endodontics
doctor of dental medicine
hospital dentistry
dental medicine, periodontics
general dentistry, orthodontics, oral surgery,
hospital dentistry, orthodontics, endodontics, periodontics
cosmetic/aesthetics dentistry, orthodontics
general and cosmetic dentistry
dental and dermatology
general dentistry, minor oral surgery, orthodontics
general dentistry, orthodontics and esthetic dentistry
dental hygiene
dental and dermatology	
dentistry-orthodontics
orthodontic/general dentistry, surgery, and tmj
general dentistry, orthodontics , endodontics, oral surgery
general d

In [75]:
# set to general dentistry
new_string = []
for each in doctors['mainspecialty']:
    if 'dent' in each:
        appendEach = True
        for each2 in invalid_dentist:
            if each == each2:
                appendEach = False
                new_string.append(each)
                break
        if appendEach:
            new_string.append('general dentistry')
    else:
        new_string.append(each)

In [76]:
# reset the indices
doctors['mainspecialty'] = new_string
specialty_list = doctors['mainspecialty'].value_counts().reset_index().rename({'index':'specialty', 'mainspecialty':'count'},axis=1)
specialty_list

,specialty,count
0,none,33283
1,general medicine,5644
2,internal medicine,5445
3,pediatrics,2590
4,obstetrics and gynecology,2077
...,...,...
1885,welder pipe filter,1
1886,"interventional radiology, vascular and interve...",1
1887,aesthetic and anti-aging medicine,1
1888,"ear, nose throat-head and neck surgery",1


In [77]:
# infer general surgery
for each in specialty_list['specialty']:
    if ('general' == each[:7] and 'surgery' in each) or each == 'surgery':
        print(each)
        print('-----')

surgery
-----
general surgery
-----
general and cancer surgery
-----
general and laparoscopic surgery
-----
general, cancer and laparoscopic surgery
-----
general  surgery
-----
general, laparoscopic and cancer surgery
-----
general laparoscopic and cancer surgery
-----
general, cancer, and laparoscopic surgery
-----
general surgery and laparoscopic surgery
-----
general and minimally invasive surgery
-----
general surgery and laparoscopy
-----
general surgery, general practice
-----
general surgery; general medicine
-----
general ears, nose and throat diseases; otolaryngology-head and neck surgery
-----
general surgery/ tcvs
-----
general surgery, aesthetic medicine
-----
general surgery (cancer, colorectal, breast, and laparoscopic surgery)
-----
general surgery, surgical oncology
-----
general cosmetic surgery
-----
general  and cancer surgery laparocopic surgery
-----
general surgery hepato-pancreato-biliary laparoscopy
-----
general surgery (thyroid, breast, gallbladder, hernia, a

In [78]:
# set to surgery
new_string = []
for each in doctors['mainspecialty']:
    if ('general' == each[:7] and 'surgery' in each) or each == 'surgery':
        new_string.append('surgery')
    else:
        new_string.append(each)

In [79]:
# reset the indices
doctors['mainspecialty'] = new_string
specialty_list = doctors['mainspecialty'].value_counts().reset_index().rename({'index':'specialty', 'mainspecialty':'count'},axis=1)
specialty_list

,specialty,count
0,none,33283
1,general medicine,5644
2,internal medicine,5445
3,pediatrics,2590
4,obstetrics and gynecology,2077
...,...,...
1820,sorry wrong sign up,1
1821,patienr,1
1822,breast surgery,1
1823,"endocrinology, diabetes mellitus, thyroid dise...",1


In [80]:
# find student
for each in specialty_list['specialty']:
    if 'student' in each:
        print(each)

student
medical student
students
med student
med-student


In [81]:
# set students to none
new_string = []
for each in doctors['mainspecialty']:
    if 'student' in each:
        new_string.append('none')
    else:
        new_string.append(each)

In [82]:
# reset the indices
doctors['mainspecialty'] = new_string
specialty_list = doctors['mainspecialty'].value_counts().reset_index().rename({'index':'specialty', 'mainspecialty':'count'},axis=1)
specialty_list

,specialty,count
0,none,33302
1,general medicine,5644
2,internal medicine,5445
3,pediatrics,2590
4,obstetrics and gynecology,2077
...,...,...
1815,geriartic,1
1816,registered pharmacist,1
1817,"organ transplant and vascular surgery, general...",1
1818,haematology,1


In [83]:
# infer psychology
for each in specialty_list['specialty']:
    if 'psychol' in each[:7]:
        print(each)

psychology
psychologist
psychology, mental health
psychology and psychiatry
psychologys
psychological first-aid
psychology, psychotherapy, eap, counselling
psychological counseling and therapy
psychology assessment


In [84]:
# set to psychology
new_string = []
for each in doctors['mainspecialty']:
    if 'psychol' in each[:7]:
        new_string.append('psychology')
    else:
        new_string.append(each)

In [85]:
# reset the indices
doctors['mainspecialty'] = new_string
specialty_list = doctors['mainspecialty'].value_counts().reset_index().rename({'index':'specialty', 'mainspecialty':'count'},axis=1)
specialty_list

,specialty,count
0,none,33302
1,general medicine,5644
2,internal medicine,5445
3,pediatrics,2590
4,obstetrics and gynecology,2077
...,...,...
1807,"endocrinology, diabetes mellitus, thyroid dise...",1
1808,reumathology,1
1809,dematology,1
1810,welder pipe filter,1


In [86]:
# infer more ENT
for each in specialty_list['specialty']:
    if 'otol' in each or 'otor' in each:
        print(each)

otorhinolaryngology - head and neck surgery
otorhinolaryngology
otolaryngology - head and neck surgery
otorhinolaryngology-head and neck surgery
otolaryngology
otorhinolaryngology head and neck surgery
otolaryngology-head and neck surgery
otolaryngology head and neck surgery
otorhinolaryngology, head and neck surgery
otorhinolaryngology (ent)
otorhinolaryngology- head and neck surgery
otolaryngology- head and neck surgery
otorhinolaryngology head and neck surgery (ent)
otorhinolaryngology - head and neck surgery (ent)
otolaryngology (ent)
otorhinolaryngology (ent) - head and neck surgery
otorhinolaryngologist
otolaryngology, head and neck surgery
otolaryngology (ear, nose, throat) - head and neck surgery
otolaryngology ( ears, nose, throat- head and neck surgery)
otolaryngology (ears, nose and throat) head and neck surgery
otolaryngology head and neck surgery (ent)
ent, otorhinolaryngology, head and neck surgery
otolaryngology heand neck surgery
radiation oncology / otorhinolaryngology

In [87]:
# set to ent
new_string = []
for each in doctors['mainspecialty']:
    if 'otol' in each or 'otor' in each:
        new_string.append('ent')
    else:
        new_string.append(each)

In [88]:
# reset the indices
doctors['mainspecialty'] = new_string
specialty_list = doctors['mainspecialty'].value_counts().reset_index().rename({'index':'specialty', 'mainspecialty':'count'},axis=1)
specialty_list

,specialty,count
0,none,33302
1,general medicine,5644
2,internal medicine,5445
3,pediatrics,2590
4,obstetrics and gynecology,2077
...,...,...
1752,"eyes,ears,nose,throat",1
1753,nutritional family practice,1
1754,sorry wrong sign up,1
1755,patienr,1


In [90]:
# specialty_list.to_excel('specialty.xlsx')

In [91]:
# read modified
specialty_list = pd.read_excel('specialty.xlsx')

In [92]:
# info on specialty_list
specialty_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1754 entries, 0 to 1753
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  1341 non-null   float64
 1   specialty   1341 non-null   object 
 2   count       1341 non-null   float64
 3   inferred    1341 non-null   object 
dtypes: float64(2), object(2)
memory usage: 54.9+ KB


In [93]:
# dropping the unnamed column
specialty_list = specialty_list.drop('Unnamed: 0', axis=1)
specialty_list

,specialty,count,inferred
0,none,33302.0,1
1,general medicine,5644.0,1
2,internal medicine,5445.0,1
3,pediatrics,2590.0,1
4,obstetrics and gynecology,2077.0,1
...,...,...,...
1749,anesthesiologiy,1.0,anesthesiology
1750,"plastic, reconstructive and aesthetic/cosmetic...",1.0,plastic surgery
1751,oral sugery,1.0,periodontics
1752,"eyes,ears,nose,throat",1.0,ent


In [94]:
# inferred is nan and actual specialty is nan
specialty_list[(specialty_list['inferred'].astype(str) == 'nan') & (specialty_list['specialty'].astype(str) != 'nan')]

,specialty,count,inferred


In [95]:
# inferred is not nan but actual specialty is nan
specialty_list[(specialty_list['inferred'].astype(str) != 'nan') & (specialty_list['specialty'].astype(str) == 'nan')]

,specialty,count,inferred


In [96]:
# inferred is nan but actual specialty is not nan
specialty_list[(specialty_list['inferred'].astype(str) == 'nan') & (specialty_list['count'].astype(str) != 'nan')]

,specialty,count,inferred


In [97]:
specialty_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1754 entries, 0 to 1753
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   specialty  1341 non-null   object 
 1   count      1341 non-null   float64
 2   inferred   1341 non-null   object 
dtypes: float64(1), object(2)
memory usage: 41.2+ KB


In [98]:
# drop the nan from the list
specialty_list = specialty_list.dropna()
specialty_list.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1341 entries, 0 to 1753
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   specialty  1341 non-null   object 
 1   count      1341 non-null   float64
 2   inferred   1341 non-null   object 
dtypes: float64(1), object(2)
memory usage: 41.9+ KB


In [99]:
# get all inferred categories
category = list(specialty_list[specialty_list['inferred'] == 1]['specialty'])
category

['none',
 'general medicine',
 'internal medicine',
 'pediatrics',
 'obstetrics and gynecology',
 'surgery',
 'family medicine',
 'dermatology',
 'ophthalmology',
 'general dentistry',
 'orthopedic surgery',
 'ent',
 'neurology',
 'cardiology',
 'psychiatry',
 'occupational medicine',
 'urology',
 'anesthesiology',
 'emergency medicine',
 'physical medicine and rehabilitation',
 'radiology',
 'psychology',
 'gastroenterology',
 'allergy',
 'endocrinology',
 'pulmonology',
 'nephrology',
 'infectious disease',
 'plastic surgery',
 'medical oncology',
 'public health',
 'optometry',
 'aesthetic medicine',
 'lifestyle medicine',
 'nursing',
 'endocrinologist',
 'veterinary medicine',
 'integrative medicine',
 'rheumatology',
 'hematology',
 'pathology',
 'nuclear medicine',
 'psychotherapy',
 'aesthetics',
 'sleep medicine',
 'pain management',
 'chiropractic',
 'nurse',
 'naturopathic medicine',
 'pain medicine',
 'geriatric medicine',
 'midwife',
 'periodontics',
 'colorectal surgery',


In [100]:
# creating a dictionary of the categrries
category_dict = {each:[] for each in category}
for i, each in specialty_list.iterrows():
    inferred = each['inferred']
    try:
        category_dict[each['specialty'] if inferred == 1 else inferred].append(each['specialty'])
    except:
        category_dict[inferred] = []

In [101]:
for each in category_dict:
    while len(category_dict[each]) != 137:
        category_dict[each].append(None)

In [102]:
# pd.DataFrame(category_dict).to_excel('categories.xlsx')

In [103]:
new_category = pd.read_excel('categories.xlsx')

In [104]:
# set the new inferred specialties
new_specialty = []
for each in doctors['mainspecialty']:
    isNone = True
    for category in new_category:
        if each in list(new_category[category]):
            isNone = False
            new_specialty.append(category)
            break
    if isNone:
        print(each)
        new_specialty.append('none')

docmark_coli@yahoo.com
software developer
entrepreneurship
internal blah
wqeqwe
medical informatics
main specialty
test
programmer
man
things
abc
bbbb
abbb
super saiyan
isiwiwiw
test
main
astronomer
paladin
abc
main specialty
ggg
hospital management
solsee
main
leading you to true health
test
aaaaaaaaaaa
electronic
test
test
main
mains
1231
123
test
human resource management
microblading
commercial
xxx
testing
it for doctors
medical officer
logistics
test
logistics
system analysis
intern
logistics
testing
post grad intern
main
f?s????e?ape?t??
test
event coordinator
mainspecialty
kirurgji
zxczcxcxzcxz
abcd
rrr
idk
a/a
conference
secretary
cabs
specialty
mha
aoeu
ooooo
'><img src=x onerror=prompt(1)>
rn/a
test
pgi
123123
gaga
retired
aaaaaaaaaaa
aaa
omomi
afsf
information technology
it specialist
loves kids
bed making
medical services
medical services
quality services
hjj
administration
com
nope
dvm
adasd
degree
aaa
lkjsadlfkj
kdsjflkfdsjg
asistant
cfvhgbjnk
med sec
me :)
breeding uropl

In [105]:
len(new_specialty)

60024

In [106]:
doctors['mainspecialty'] = new_specialty
specialty_list = doctors['mainspecialty'].value_counts().reset_index().rename({'index':'specialty', 'mainspecialty':'count'},axis=1)
specialty_list

,specialty,count
0,none,33949
1,general medicine,6057
2,internal medicine,5516
3,pediatrics,2629
4,obstetrics and gynecology,2126
...,...,...
66,audiology,2
67,laboratory medicine,2
68,medical music therapy,1
69,hyperbaric medicine,1


Checking ID Validity

In [231]:
# check if all ids are valid
doctors[[(len(each) != 32) for each in doctors['doctorid']]]

,doctorid,mainspecialty,age


Checking Age Validity

In [15]:
doctors[doctors['age'].notna()].sort_values('age')

,doctorid,mainspecialty,age
10403,5B7FCE29C95E235AEF97EE358962A3CA,Pediatrics,4.0
1532,495DABFD0CA768A3C3ABD672079F48B6,Internal Medicine,7.0
1521,D88518ACBCC3D08D1F18DA62F9BB26EC,internal medicine,7.0
1518,D72FBBCCD9FE64C3A14F85D225A046F4,general surgery,7.0
1638,A51C896C9CB81ECB5A199D51AC9FC3C5,cardio,7.0
...,...,...,...
35035,C726062049174DD685BBB960958FA1C1,None,132.0
318,F73B76CE8949FE29BF2A537CFA420E8F,Family medicines,160.0
59136,B02EE9D17C3B943AFFA39F55282793FC,"Psychology, Psychotherapy, EAP, Counselling",938.0
31154,88D619AD97413B44D6659073B7B12F32,OB,944.0


In [529]:
# if age is nan, or negative, or greater than 120, set it to nan
new_age = []
for each in doctors['age']:
    if str(each) == 'nan' or each < 0.0 or each > 120:
        new_age.append(np.nan)
    else:
        new_age.append(each)

In [532]:
# save the new age values
doctors['age'] = new_age

### Cleaning px.csv

Exploration

In [170]:
# preview
px

,pxid,age,gender
0,5A1718EC380AFE6BE24D63EE78CDA043,NaN,FEMALE
1,C0FC9D6384C7F579F1048A461C298B8C,8.0,FEMALE
2,FB0C4098E2F7FD0BC3865382242E7034,2.0,FEMALE
3,EB3C1CBD5A2AC52C69BE0B90C5E149A7,54.0,MALE
4,92C92C2EB4B51FBB3CD354165BA5F027,67.0,FEMALE
...,...,...,...
6507808,9D8045364678C651016538B78784720E,56,MALE
6507809,BCC86927CB7F687624859D5B7C9AB8B6,60,MALE
6507810,2890460EE1FD505905A0A55834EBDC06,17,FEMALE
6507811,3989D63FA4DD7C54911AA085CB7CF38A,62,MALE


In [171]:
# info about patients
px.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6507813 entries, 0 to 6507812
Data columns (total 3 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   pxid    object
 1   age     object
 2   gender  object
dtypes: object(3)
memory usage: 149.0+ MB


In [172]:
# counts of each gender
px['gender'].value_counts()

FEMALE    3642569
MALE      2865243
gender          1
Name: gender, dtype: int64

Removing Duplicates

In [173]:
# check if there are duplicated ids
print("Duplicate IDs:", px['pxid'].duplicated(keep=False).any())

Duplicate IDs: True


In [175]:
# see the duplicated ids
px[px['pxid'].duplicated(keep=False)]

,pxid,age,gender
0,5A1718EC380AFE6BE24D63EE78CDA043,NaN,FEMALE
1,C0FC9D6384C7F579F1048A461C298B8C,8.0,FEMALE
2,FB0C4098E2F7FD0BC3865382242E7034,2.0,FEMALE
3,EB3C1CBD5A2AC52C69BE0B90C5E149A7,54.0,MALE
4,92C92C2EB4B51FBB3CD354165BA5F027,67.0,FEMALE
...,...,...,...
1990656,4EDAEEA20EA10C4DB4876E737CEAD0DA,7.0,FEMALE
3959871,D8E7A835F6418A9A8ABA99A87390446D,38.0,MALE
3959872,D8E7A835F6418A9A8ABA99A87390446D,38.0,MALE
3960066,FBA46EA3EF7CCD4F3551C22272FE865F,42.0,MALE


Checking valid ages

In [176]:
# set the age to float if possible
for each in px['age']:
    try:
        float(each)
    except:
        print(each)

age


In [177]:
# find the row where the age is 'age'
px[px['age'] == 'age']

,pxid,age,gender
995328,pxid,age,gender


In [178]:
# drop that row
px = px.drop(995328)

In [179]:
# reset the indices
px = px.reset_index().drop('index', axis=1)

In [180]:
# setting the ages as floats
px['age'] = px['age'].astype(float)

In [181]:
# if ages are nan, less than 0, or greater than 120, the age is set to nan
new_age = []
for each in px['age']:
    if str(each) == 'nan' or each < 0.0 or each > 120:
        new_age.append(np.nan)
    else:
        new_age.append(each)

In [182]:
# the new age values are saved
px['age'] = new_age

In [183]:
# preview of patients in descending age values
px=px.sort_values('age',ascending=False).reset_index().drop('index',axis=1)
px

,pxid,age,gender
0,ABDA73C55BAB2F20B30BD63045EEC0E8,120.0,FEMALE
1,2789C09850D7A504FD0E6BB0B887B119,120.0,MALE
2,093B9B81F9965A23B6ABE609615DDFAD,120.0,FEMALE
3,AD65691508EE5B6B64EE33F29E5181F9,120.0,FEMALE
4,312A4DBFC9997ACAA08AB3F1CD0ED12A,120.0,FEMALE
...,...,...,...
6507807,E4A2C6A059BD27024843AEA8924ACA01,NaN,MALE
6507808,91A193DBF9891D001A11C6ED9093F2F6,NaN,FEMALE
6507809,115BB28CFD4701B1E72CA436412D2A34,NaN,MALE
6507810,C53E03E6794972AB45656853357AC65A,NaN,MALE


In [202]:
# if there are duplicated ids, get their older age
px = px.drop_duplicates(subset='pxid', keep='first')

In [203]:
# check if there are still duplicates
px['pxid'].duplicated(keep=False).any()

False

In [204]:
px

,pxid,age,gender
0,ABDA73C55BAB2F20B30BD63045EEC0E8,120.0,FEMALE
1,2789C09850D7A504FD0E6BB0B887B119,120.0,MALE
2,093B9B81F9965A23B6ABE609615DDFAD,120.0,FEMALE
3,AD65691508EE5B6B64EE33F29E5181F9,120.0,FEMALE
4,312A4DBFC9997ACAA08AB3F1CD0ED12A,120.0,FEMALE
...,...,...,...
6507807,E4A2C6A059BD27024843AEA8924ACA01,NaN,MALE
6507808,91A193DBF9891D001A11C6ED9093F2F6,NaN,FEMALE
6507809,115BB28CFD4701B1E72CA436412D2A34,NaN,MALE
6507810,C53E03E6794972AB45656853357AC65A,NaN,MALE


Checking for valid ids

In [205]:
# check if all ids are valid
px[[(len(each) != 32) for each in px['pxid']]]

,pxid,age,gender


In [209]:
# preview
px = px.reset_index().drop('index', axis=1)
px

,pxid,age,gender
0,ABDA73C55BAB2F20B30BD63045EEC0E8,120.0,FEMALE
1,2789C09850D7A504FD0E6BB0B887B119,120.0,MALE
2,093B9B81F9965A23B6ABE609615DDFAD,120.0,FEMALE
3,AD65691508EE5B6B64EE33F29E5181F9,120.0,FEMALE
4,312A4DBFC9997ACAA08AB3F1CD0ED12A,120.0,FEMALE
...,...,...,...
5512477,E4A2C6A059BD27024843AEA8924ACA01,NaN,MALE
5512478,91A193DBF9891D001A11C6ED9093F2F6,NaN,FEMALE
5512479,115BB28CFD4701B1E72CA436412D2A34,NaN,MALE
5512480,C53E03E6794972AB45656853357AC65A,NaN,MALE


### Cleaning appointments.csv

Getting valid appointments only (based on id) 

In [228]:
# remove appointments with non-existing patient ids
appointments = appointments[appointments['pxid'].isin(px['pxid'])].reset_index().drop('index', axis=1)

In [229]:
# remove appointments with non-existing clinic ids
appointments = appointments[appointments['clinicid'].isin(clinics['clinicid'])]

In [232]:
# remove appointments with non-existing doctor ids
appointments = appointments[appointments['doctorid'].isin(doctors['doctorid'])]

In [233]:
# check
appointments

,pxid,clinicid,doctorid,apptid,status,TimeQueued,QueueDate,StartTime,EndTime,type,Virtual
0,FE4A5D5A20EC492D2FC691F126A568AB,3B8D83483189887A2F1A39D690463A8F,ACB3A881C7CE9ABCAE0CE8C99C86A906,04BC9218E072BEEFEBEE9C97B78A35C9,Queued,2020-06-22 05:11:38.247000000,2020-07-02 16:00:00,2020-07-02 17:00:38.073000000,2020-07-02 17:15:38.073000000,Consultation,NaN
1,E6BC8E80685AA4A239617F196F12A0C3,3B8D83483189887A2F1A39D690463A8F,ACB3A881C7CE9ABCAE0CE8C99C86A906,F2335AAFB7EE079C7E7306591057C0C9,Queued,2020-06-26 04:51:16.403000000,2020-07-12 16:00:00,2020-07-12 17:00:15.833000000,2020-07-12 17:15:15.833000000,Consultation,NaN
2,E430B3D3AEED550612FFDBD0F2F94112,3B8D83483189887A2F1A39D690463A8F,ACB3A881C7CE9ABCAE0CE8C99C86A906,B2C83D6DE89A88C5CD9EA1FD5FC00602,Queued,2020-07-13 07:00:50.127000000,2020-07-19 16:00:00,2020-07-20 05:00:49.687000000,2020-07-20 05:15:49.687000000,Consultation,NaN
3,E6BC8E80685AA4A239617F196F12A0C3,3B8D83483189887A2F1A39D690463A8F,ACB3A881C7CE9ABCAE0CE8C99C86A906,DA4023B5A8C3F1BD540EA82552F21134,Queued,2020-07-13 06:07:13.537000000,2020-07-28 16:00:00,2020-07-29 05:00:13.133000000,2020-07-29 05:15:13.133000000,Consultation,NaN
4,41E3F930274A7704305EE197F3434877,3B8D83483189887A2F1A39D690463A8F,ACB3A881C7CE9ABCAE0CE8C99C86A906,C805F7992F18E33AB85CEBD572680943,Queued,2020-06-26 05:54:35.117000000,2020-09-27 16:00:00,2020-09-27 17:00:34.917000000,2020-09-27 17:15:34.917000000,Consultation,NaN
...,...,...,...,...,...,...,...,...,...,...,...
320135,C262D44C1E0C8451E3328343BE666813,2D5B3BE8503916AF8D41E5ADD640B0B6,F7F07E7DAB09533BC71247A5B29A7373,3E30C5A14D659B05E1AB506479538996,Queued,2021-03-16 22:28:27.627000000,2021-03-16 16:00:00,2021-03-16 22:26:00,2021-03-19 06:25:00,Inpatient,NaN
320136,F5D5EDE903850C3B541AB19AD4D21315,2D5B3BE8503916AF8D41E5ADD640B0B6,F7F07E7DAB09533BC71247A5B29A7373,BA7DC84EDA21C6D219CAB7B782876188,Queued,2021-03-31 04:21:44.753000000,2021-03-30 16:00:00,2021-03-31 04:19:00,2021-03-31 21:33:00,Inpatient,NaN
320137,838B6A62B7DC186CDECF5D6564EFC8CB,2D5B3BE8503916AF8D41E5ADD640B0B6,F7F07E7DAB09533BC71247A5B29A7373,167209C9AD16B30A73E90F3EAA5C10CA,Queued,2021-05-16 11:01:32.287000000,2021-05-13 16:00:00,2021-05-14 11:00:00,2021-05-19 05:04:00,Inpatient,NaN
320138,BAF981CE16CC723515D127A34F4484F0,2D5B3BE8503916AF8D41E5ADD640B0B6,F7F07E7DAB09533BC71247A5B29A7373,11C959D3644700F43963D8B7458A0F56,Queued,2021-09-21 23:45:20.367000000,2021-09-21 16:00:00,2021-09-21 23:44:00,2021-09-24 06:29:00,Inpatient,NaN


Setting DateTime datatypes accordingly

In [235]:
# check the TimeQueued variable if convertible to datetime datatype
pd.to_datetime(appointments['TimeQueued']).sort_values()

208663   1971-12-09 16:00:00.000
3045     2000-07-14 16:00:00.000
177485   2004-05-11 16:00:00.000
177486   2004-05-11 16:00:00.000
177487   2004-05-11 16:00:00.000
                   ...          
313513   2024-01-23 09:09:06.470
241679   2024-01-23 09:13:51.657
305741   2024-01-23 09:16:56.330
300990   2024-01-23 10:00:00.493
171537   2024-01-23 13:14:45.903
Name: TimeQueued, Length: 320140, dtype: datetime64[ns]

In [236]:
# check the StartTime variable if convertible to datetime datatype
pd.to_datetime(appointments['StartTime']).sort_values()

208663   1971-12-10 05:04:00
3045     2000-07-15 02:15:22
177492   2004-05-11 16:00:00
177491   2004-05-11 16:00:00
177485   2004-05-11 16:00:00
                 ...        
319717                   NaT
319719                   NaT
319723                   NaT
319730                   NaT
320050                   NaT
Name: StartTime, Length: 320140, dtype: datetime64[ns]

In [237]:
# check the QueueDate variable if convertible to datetime datatype
pd.to_datetime(appointments['QueueDate']).sort_values()

208663   1971-12-09 16:00:00
3045     2000-07-14 16:00:00
177485   2004-05-11 16:00:00
177486   2004-05-11 16:00:00
177487   2004-05-11 16:00:00
                 ...        
125185   2025-01-20 16:00:00
130906   2025-01-20 16:00:00
125186   2025-06-14 16:00:00
142991   2026-01-05 16:00:00
1569     2027-03-05 16:00:00
Name: QueueDate, Length: 320140, dtype: datetime64[ns]

In [239]:
# check the EndTime variable if convertible to datetime datatype
pd.to_datetime(appointments['EndTime']).sort_values()

203452   2017-09-10 22:00:00
207226   2017-09-14 22:15:00
207227   2017-09-28 22:15:00
207230   2017-10-17 17:12:00
202791   2017-11-08 08:00:00
                 ...        
319723                   NaT
319730                   NaT
319731                   NaT
320050                   NaT
320093                   NaT
Name: EndTime, Length: 320140, dtype: datetime64[ns]

In [241]:
# convert to datetime format
appointments['TimeQueued'] = pd.to_datetime(appointments['TimeQueued'])
appointments['StartTime'] = pd.to_datetime(appointments['StartTime'])
appointments['QueueDate'] = pd.to_datetime(appointments['QueueDate'])
appointments['EndTime'] = pd.to_datetime(appointments['EndTime'])

Checking other values

In [242]:
# check status
appointments['status'].value_counts()

Queued      179638
Complete    135259
Cancel        2258
Serving       1544
NoShow        1008
Skip           433
Name: status, dtype: int64

In [243]:
# check type
appointments['type'].value_counts()

Consultation    319931
Inpatient          209
Name: type, dtype: int64

In [244]:
# check virtual
appointments['Virtual'].value_counts()

False    124157
True       5847
Name: Virtual, dtype: int64

# Load

Save Progress

In [53]:
division = len(px) // 8
for i in range(8):
    if i != 7:
        px.loc[division*i + 1:division*(i+1)].to_parquet(f"cleaned/px_{i+1}.parquet")
    else:
        px.loc[division*i + 1:].to_parquet(f"cleaned/px_{i+1}.parquet")
doctors.to_parquet('cleaned/doctors.parquet')
clinics.to_parquet('cleaned/clinics.parquet')
appointments.to_parquet('cleaned/appointments.parquet')

Load Progress

In [55]:
doctors = pd.read_parquet('cleaned/doctors.parquet')
clinics = pd.read_parquet('cleaned/clinics.parquet')
px = pd.DataFrame()
for i in range(8):
    temp = pd.read_parquet(f"cleaned/px_{i+1}.parquet")
    px = pd.concat([px,temp])
appointments = pd.read_parquet('cleaned/appointments.parquet')

Connect to MySQL

In [245]:
host = '127.0.0.1'
user = 'root'
password = '12345678'
port = '3306'
db_name = 'mco1_warehouse'
engine = db.create_engine('mysql+pymysql://' + user + ':' + password + '@' + host + ':' + port + '/' + db_name)

Define the schemas of CSV files

In [256]:
# clinics schema
clinics_schema = {
    "clinicid":db.types.VARCHAR(32),
    "hospitalname":db.types.VARCHAR(255),
    "IsHospital":db.types.BOOLEAN(),
    "City":db.types.VARCHAR(255),
    "Province":db.types.VARCHAR(255),
    "RegionName":db.types.VARCHAR(255)
}

In [257]:
# doctors schema
doctors_schema = {
    "doctorid":db.types.VARCHAR(32),
    "mainspecialty":db.types.VARCHAR(255),
    "age":db.dialects.mysql.TINYINT(255),
}

In [258]:
# px schema
px_schema = {
    "pxid":db.types.VARCHAR(32),
    "age":db.dialects.mysql.TINYINT(255),
    "gender":db.types.Enum("MALE", "FEMALE")
}

In [259]:
# appointments schema
appointments_schema = {
    "pxid":db.types.VARCHAR(32),
    "clinicid":db.types.VARCHAR(32),
    "doctorid":db.types.VARCHAR(32),
    "apptid":db.types.VARCHAR(32),
    "status":db.types.Enum("Queued", "Complete", "Serving","Cancel","NoShow","Skip"),
    "TimeQueued":db.types.DATETIME(),
    "QueueDate":db.types.DATE(),
    "StartTime":db.types.DATETIME(),
    "EndTime":db.types.DATETIME(),
    "type":db.types.Enum("Consultation","Inpatient"),
    "Virtual":db.types.BOOLEAN()
}

Insert the data to warehouse

In [260]:
# create table if doesn't exist
with engine.connect() as conn:
    conn.execute(db.text("""
        DROP SCHEMA IF EXISTS `mco1_warehouse`;
    """))
    conn.execute(db.text("""
        CREATE SCHEMA `mco1_warehouse`;
    """))

In [261]:
engine = db.create_engine('mysql+pymysql://' + user + ':' + password + '@' + host + ':' + port + '/' + db_name)

In [262]:
appointments.to_sql('appointments', engine, if_exists='replace', index=False, dtype=appointments_schema)

320140

In [263]:
doctors.to_sql('doctors', engine, if_exists='replace', index=False, dtype=doctors_schema)

60024

In [264]:
clinics.to_sql('clinics', engine, if_exists='replace', index=False, dtype=clinics_schema)

53962

In [265]:
px.to_sql('px', engine, if_exists='replace', index=False, dtype=px_schema)

5512482

Set primary keys

In [2]:
with engine.connect() as conn:
    conn.execute(db.text("""
        ALTER TABLE `mco1_warehouse`.`px` 
        CHANGE COLUMN `pxid` `pxid` VARCHAR(32) NOT NULL ,
        ADD PRIMARY KEY (`pxid`);
    """))
    print("pxid IS A PRIMARY KEY IN PX")
    conn.execute(db.text("""
        ALTER TABLE `mco1_warehouse`.`clinics` 
        CHANGE COLUMN `clinicid` `clinicid` VARCHAR(32) NOT NULL ,
        ADD PRIMARY KEY (`clinicid`);
    """))
    print("clinicid IS A PRIMARY KEY IN CLINICS")
    conn.execute(db.text("""
        ALTER TABLE `mco1_warehouse`.`doctors` 
        CHANGE COLUMN `doctorid` `doctorid` VARCHAR(32) NOT NULL ,
        ADD PRIMARY KEY (`doctorid`);
    """))
    print("doctorid IS A PRIMARY KEY IN DOCTORS")
    conn.execute(db.text("""
        ALTER TABLE `mco1_warehouse`.`appointments` 
        CHANGE COLUMN `apptid` `apptid` VARCHAR(32) NOT NULL ,
        ADD PRIMARY KEY (`apptid`);
    """))
    print("apptid IS A PRIMARY KEY IN APPOINTMENTS")

Set foreign keys

In [1]:
with engine.connect() as conn:
    conn.execute(db.text("""
        ALTER TABLE `mco1_warehouse`.`appointments` 
        ADD CONSTRAINT `fk_px`
        FOREIGN KEY (`pxid`)
        REFERENCES `mco1_warehouse`.`px` (`pxid`);
    """))
    print("pxid IS A FOREIGN KEY IN APPOINTMENTS")
    conn.execute(db.text("""
        ALTER TABLE `mco1_warehouse`.`appointments` 
        ADD CONSTRAINT `fk_clinics`
        FOREIGN KEY (`clinicid`)
        REFERENCES `mco1_warehouse`.`clinics` (`clinicid`);
    """))
    print("clinicid IS A FOREIGN KEY IN APPOINTMENTS")
    conn.execute(db.text("""
        ALTER TABLE `mco1_warehouse`.`appointments` 
        ADD CONSTRAINT `fk_doctors`
        FOREIGN KEY (`doctorid`)
        REFERENCES `mco1_warehouse`.`doctors` (`doctorid`);
    """))
    print("doctorid IS A FOREIGN KEY IN APPOINTMENTS")